# Tutorial 2: Conformance Checking (Order-to-Cash)

This tutorial demonstrates how to perform conformance checking to compare a process model with an event log for an order-to-cash (O2C) process.

## 1. Import necessary libraries

In [ ]:
import pandas as pd
from erp_processminer.eventlog.serialization import dataframe_to_log
from erp_processminer.discovery.heuristics_miner import discover_petri_net_with_heuristics
from erp_processminer.conformance.token_replay import calculate_conformance
from erp_processminer.visualization.graphs import visualize_petri_net

## 2. Create a Sample Event Log

For this tutorial, we will start with an event log that has already been created. This represents a simple order-to-cash process.

In [ ]:
log_data = [
    ['C-01', 'Create Order', '2023-02-01'],
    ['C-01', 'Confirm Order', '2023-02-02'],
    ['C-01', 'Create Shipment', '2023-02-03'],
    ['C-01', 'Ship Goods', '2023-02-04'],
    ['C-01', 'Send Invoice', '2023-02-05'],
    ['C-01', 'Receive Payment', '2023-02-10'],

    ['C-02', 'Create Order', '2023-02-02'],
    ['C-02', 'Confirm Order', '2023-02-03'],
    ['C-02', 'Cancel Order', '2023-02-04'], # Deviant behavior

    ['C-03', 'Create Order', '2023-02-05'],
    ['C-03', 'Confirm Order', '2023-02-06'],
    ['C-03', 'Create Shipment', '2023-02-07'],
    ['C-03', 'Ship Goods', '2023-02-08'],
    ['C-03', 'Send Invoice', '2023-02-09'],
    ['C-03', 'Receive Payment', '2023-02-15'],
]

log_df = pd.DataFrame(log_data, columns=['case_id', 'activity', 'timestamp'])
event_log = dataframe_to_log(log_df)

print(f"Event log created with {len(event_log.traces)} traces.")

## 3. Discover a Process Model

We will use the Heuristics Miner to discover a Petri net from the event log. We'll use a relatively high dependency threshold to get a clean model.

In [ ]:
petri_net = discover_petri_net_with_heuristics(event_log, dependency_thresh=0.7)

# Visualize the discovered model
g = visualize_petri_net(petri_net, output_file='o2c_petri_net')
g

## 4. Perform Conformance Checking

Now we can use token-based replay to check the conformance of the log against the discovered model. A fitness of 1.0 means the log perfectly conforms to the model.

In [ ]:
avg_fitness, trace_results = calculate_conformance(event_log, petri_net)

print(f"Average Fitness: {avg_fitness:.3f}")

for i, res in enumerate(trace_results):
    print(f"Trace {i+1}: Fitness = {res['fitness']:.3f}, Missing = {res['missing_tokens']}, Remaining = {res['remaining_tokens']}")

The results show that the second trace, which contains the 'Cancel Order' activity, has a lower fitness score because this behavior is not well-represented in the discovered model.